## Notebook for topic-modellering av Delicia.

Litt boilerplate import

In [1]:
import topicmodelling as tm
import imp
import util
from bs4 import BeautifulSoup
import fork_kombo as fk
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline

## Definisjon av konkordans

konk(corpus, ord, n) har tre parametre, det første er korpuset (som i variabelen **corpus**), det neste er en liste av ord, skriv ordene med anførselstegn skilt med blanke som f.eks. "droit sa raison". Det siste parametret, n er et tall fra 1 og nedover til for eksempel 1/2, som angir hvor mange av ordene som skal treffes på. Hele avsnittet vises - ser at for den franske er hver setning tolket som avsnitt.

Prøv det litt og se hvor det bærer. Eksempel på bruk nederst i notebooken. Trenger du litt mindre konkordans, mer standard, så bare si fra!

In [34]:
def konk(corpus, words, n=1):
    
    """n is percent of match - jaccardish"""

    words = [w.strip() for w in words.split()]
    lines = ""
    for p in corpus:
        
        if len(set(words) & set(p)) >= n*len(set(words)):
            line = ' '.join(p)
            for w in words:
                line = line.replace(w, "<span style='color:tomato;'>{w}</span>".format(w=w))
            lines+= "<p>{konk}</p>".format(konk = line)
            #print(line)
    return HTML(lines)

### Hvor ligger tekstene

De norske kan eventuelt splittes opp i to

In [49]:
#my_norwegian_files = [ 'ad3.txt', 'constance.txt']
my_french_files = ['sand-indiana.txt']

## Kjør en av cellene under

Den første er med fransk tokenisering - skulle funke, men sjekk at alt er som det skal

### Fransk

In [51]:
# fransk tekst

corpus=[]
for filename in my_french_files:
    text = open(filename, encoding='utf-8').read()
    for paragraph in text.split('\n\n'):
        corpus.append(fk.tokenize(paragraph))

### Norsk

### Se på korpuset

For å sjekke tokenisering, at punktum og andre tegn er skilt ut.

In [52]:
# velg noen avsnitt for å få et inntrykk - 5 eller 10 om gangen.

for p in corpus[100:105]:
    print(' '.join(p))

ne dois pas vous le dire ... – Eh bien , moi , je le sais , Indiana . Je sais toute votre histoire , toute votre vie . Rien de ce qui vous concerne ne m ’ est étranger , parce que rien de ce qui vous concerne ne m ’ est indifférent . J ’ ai voulu tout connaître de vous , et je n ’ ai rien appris que ne m ’ eût révélé un instant passé chez vous , lorsqu ’ on m ’ apporta tout sanglant , tout brisé à vos pieds , et que votre mari s ’ irrita de vous voir , si belle et si bonne , me faire un appui de vos bras mœlleux , un baume de votre douce haleine . Lui , jaloux ! oh ! je le conçois bien ; à sa place , je le serais , Indiana ; ou plutôt , à sa place , je me tuerais ; car , être votre époux , madame , vous posséder , vous tenir dans ses bras , et ne pas vous mériter , n ’ avoir pas votre cœur , c ’ est être le plus misérable ou le plus lâche des hommes . – Ô ciel ! taisez-vous , s ’ écria-t- elle en lui fermant la bouche avec ses mains , taisez-vous , car vous me rendez coupable . Pourquo

In [40]:
freqs = tm.corpusfreq(corpus)

In [41]:
freqs.most_common(20)

[(',', 7346),
 ('’', 6621),
 ('de', 4375),
 ('.', 3222),
 ('et', 2649),
 ('la', 2150),
 ('le', 1734),
 ('à', 1725),
 ('l', 1673),
 (';', 1431),
 ('il', 1313),
 ('que', 1308),
 ('d', 1256),
 ('un', 1141),
 ('vous', 1099),
 ('les', 1093),
 ('qu', 949),
 ('qui', 940),
 ('ne', 935),
 ('en', 915)]

## Lag  selve stoppordliste

Ordlista kan endres ved å legge til og trekke fra. Viser nedenfor hvordan det gjøres.

Ett paramater som er enkelt å endre er tallet i **freqs.most_common()**, bare endre det for å få andre stoppordlister. Ord som er tatt ut kan legges inn igjen. Se et par celler nedenfor.

In [42]:
korpus_stopp = dict({x[0]:freqs[x[0]] for x in freqs.most_common(100)})

**Sjekk ut hva som fjernes...**

In [43]:
print(korpus_stopp.keys())

dict_keys([',', '’', 'de', '.', 'et', 'la', 'le', 'à', 'l', ';', 'il', 'que', 'd', 'un', 'vous', 'les', 'qu', 'qui', 'ne', 'en', 'pas', 'elle', 'une', 'dans', 'je', 'pour', 'se', 'lui', 'son', 's', 'n', 'était', 'sa', 'avait', 'des', 'est', 'ses', '–', '!', 'plus', 'du', 'sur', 'avec', 'ce', 'si', 'Raymon', 'me', 'Il', 'au', 'mais', 'Delmare', 'm', 'comme', 'cette', 'Indiana', '?', 'Ralph', 'tout', 'Mme', 'j', 'par', 'y', 'c', 'bien', 'sans', 'mon', 'ai', 'Je', 'a', 'femme', 'votre', 'moi', 'vie', 'homme', '...', 'Elle', 'ma', 'nous', 'ces', ':', 'cœur', 'eût', 'être', 'aux', 'on', 'dit', 'Mais', 'où', 'fait', 'cet', 'encore', 'M.', 'amour', 'Noun', 'autre', 'donc', 'toute', 'tous', 'Vous', 'été'])


In [44]:
tm.modeller_set(corpus, tema=10, chunk_size=100, stop_words = korpus_stopp, topic_size=15)

['monde, peu, faire, toutes, aussi, mère, Ramière, esprit, point, fut, voulait, rien, pouvait, jamais, quand',
 'tu, te, ton, t, toi, ta, suis, tes, mes, as, Et, serai, es, sais, sang',
 'avez, suis, vos, mes, jamais, J, dit-elle, êtes, aime, peut-être, jour, puis, Et, car, ici',
 'colonel, sir, monsieur, madame, faire, répondit, suis, Ramière, ici, jeune, dit-il, main, mari, Le, toujours',
 'leur, ils, leurs, deux, avaient, Les, étaient, Ils, toujours, sont, quelques, main, eux, ont, hommes',
 'ni, sais, réponse, mal, fois, ordinaire, caractère, espoir, Dieu, pensées, Comme, vers, personnage, première, mes',
 'sous, dont, vers, chambre, porte, même, instant, tête, fit, avaient, devant, La, yeux, fois, fut',
 'ou, quelque, yeux, chez, car, chose, point, là, aimait, peut-être, sir, courage, quand, esprit, puis',
 'aurais, mes, vos, J, étais, voir, bras, car, seul, souffrir, pieds, ans, sourire, vôtre, quand',
 '«, », Dieu, avoir, jour, peut-être, même, J, C, fut, avais, âme, car, jamais

** Her er noen konkordanser også **

In [48]:
konk(corpus, 'homme amour enfant', 1)